In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
%matplotlib inline

import sys
import os
import yaml
import itertools
sys.path.append('../')
from utils.utils import ModelLoader,DepLoader
from matplotlib import pyplot as plt
from timeit import timeit  

In [2]:
with open('../config/rule_shortdeck_with_holdemrule.yaml') as fhdl:
    cfg = yaml.load(fhdl)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [3]:
env = DepLoader(cfg)

In [4]:
depth = 6
json = env.tree_builder.gen_km_json("../imgs/tree_depth{}.km".format(depth),limit=depth,ret_json=True)

# 查看游戏总状态数

In [5]:
hiss = []
roots = []
def count_dealcard(root,depth=0):
    if root.children is not None:
        num = 0
        for one_child in root.children.values():
            num += count_dealcard(one_child)
        return num
    else:
        if 'terminal' in root.to_string():
            hiss.append(root.bet_history)
            roots.append(root)
        return 0 if 'terminal' in root.to_string() else 1

In [6]:
count_dealcard(env.tree_builder.root)

47662

In [7]:
%timeit count_dealcard(env.tree_builder.root)

891 ms ± 2.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%lprun -f count_dealcard count_dealcard(env.tree_builder.root)

Timer unit: 1e-06 s

Total time: 1.82193 s
File: <ipython-input-5-3dbdc1fb9032>
Function: count_dealcard at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def count_dealcard(root,depth=0):
     4    156709        83402      0.5      4.6      if root.children is not None:
     5     61385        23555      0.4      1.3          num = 0
     6    218093       115805      0.5      6.4          for one_child in root.children.values():
     7    156708        76103      0.5      4.2              num += count_dealcard(one_child)
     8     61385        22854      0.4      1.3          return num
     9                                               else:
    10     95324       751080      7.9     41.2          if 'terminal' in root.to_string():
    11     47662        28820      0.6      1.6              hiss.append(root.bet_history)
    12     47662        23456      0.5      1.3              roots.append(root)
    13  

In [9]:
len(hiss)

476620

# 查看叶子节点的bet history

In [10]:
index = 5000 + 1
print(hiss[index])
print(roots[index].committed)

['call', 'check', 'check', 'bet_1', 'raise_2', 'raise_5', 'call', 'check', 'bet_7', 'raise_21', 'raise_84', 'call', 'check', 'bet_2', 'fold']
[100.0, 98.0]


# 查看card abstraction 工作状态

In [18]:
cards_sampled = env.card_sampler.sample_card(4)
print(cards_sampled)

['Qc', 'Ah', '6s', 'As']


In [19]:
env.card_abstraction.abstract(cards_sampled)

'QcAh6sAs'

In [20]:
# TODO 写strategy 和trainer,修改abstraction算法
# TODO 每层都需要用不同的action choice

In [99]:
bits = int('0b11111', 2)

In [100]:
bits

31

In [101]:
def get_lexographically_next_bit_sequence( bits):
    """
    Bit hack from here:
    http://www-graphics.stanford.edu/~seander/bithacks.html#NextBitPermutation

    Generator even does this in poker order rank 
    so no need to sort when done! Perfect.
    """
    t = (bits | (bits - 1)) + 1 
    next = t | ((((t & -t) // (bits & -bits)) >> 1) - 1)  
    yield next
    while True:
        t = (next | (next - 1)) + 1 
        next = t | ((((t & -t) // (next & -next)) >> 1) - 1)
        yield next

In [102]:
gen = get_lexographically_next_bit_sequence(bits)

In [114]:
bin(next(gen))

'0b1101011'

In [76]:
t = (bits | (bits - 1)) + 1 

In [79]:
t | ((((t & -t) / (bits & -bits)) >> 1) - 1)  

47

In [80]:
(t & -t) 

32

In [81]:
(bits & -bits)

1